In [1]:
import fenics as fen
import numpy as np
from examples.rectangular_waveguide.rectangular_waveguide import RectangularWaveguide

In [2]:
Lx, Ly = 5.0, 1.0
Nx, Ny = 201, 41
m = 1
g_N = fen.Expression('sin(x[1]*m*pi/Ly)', degree=2, Ly=Ly, m=m)
WG = RectangularWaveguide(Lx=Lx, Ly=Ly, Nx=Nx, Ny=Ny, g_N=g_N)
WG.setup()

In [3]:
N = 100
A = []
omegas = np.linspace(3, 4, N)
for omega in omegas:
    WG.solve(omega)
    A_sol = WG.get_solution()
    A.append(A_sol.vector())

In [4]:
def inner_product(v, w, M=None):
    """Compute M-inner-product of two vectors v and w"""
    if M is None:
        return (v*w).sum()
    return ((M*v)*w).sum()

In [5]:
def norm(v, M=None):
    """Compute M-norm of a vector v"""
    return pow(inner_product(v, v, M), 0.5)

In [6]:
def gram_schmidt(E, M=None):
    """M-orthonormalize the elements of a list E"""
    E = [fen.Vector(e) for e in E]
    E_on = [E[0] / norm(E[0], M)]
    for i in range(1, len(E)):
        for j in range(i):
            E[i] -= inner_product(E_on[j], E[i], M) * E_on[j]
        E_on.append(E[i] / norm(E[i], M))
    return E_on

In [7]:
def get_orthonormal_vectors(V, N, M=None, seed=0):
    """Produce list of N M-orthonormal elements"""
    np.random.seed(seed)
    E = []
    for i in range(N):
        e = fen.Function(V).vector()
        e[:] = np.random.randn(e.size())
        E.append(e)
    return gram_schmidt(E, M)

In [8]:
def householder_triangularization(A, E, M=None):
    """Compute the matrix R of a QR-decomposition of A"""
    N = len(A)
    # Ultra-explicit copy assignment (cost me 5h of my life, fuck you dolfin!!!)
    A = [fen.Vector(a) for a in A]
    E = [fen.Vector(e) for e in E]
    R = np.zeros((N, N))

    for k in range(N):
        R[k, k] = norm(A[k], M)

        alpha = inner_product(E[k], A[k], M)
        if abs(alpha) > 1e-17:
            E[k] *= - alpha / abs(alpha)
  
        v = R[k, k] * E[k] - A[k]
        for j in range(k):
            v -= inner_product(E[j], v, M) * E[j]

        sigma = norm(v, M)
        if abs(sigma) > 1e-17:
            v /= sigma
        else:
            v = E[k]

        for j in range(k+1, N):
            A[j] -= 2 * v * inner_product(v, A[j], M)
            R[k, j] = inner_product(E[k], A[j], M)
            A[j] -= E[k] * R[k, j]

    return R

In [11]:
M = WG.get_M(tosparse=False)#fen.assemble(fen.dot(fen.TrialFunction(WG.V), fen.TestFunction(WG.V)) * fen.dx)
E = get_orthonormal_vectors(WG.V, len(A), M)
R = householder_triangularization(A, E, M)

In [12]:
G = np.array([[inner_product(A[i], A[j], M) for i in range(N)] for j in range(N)])
print(np.sum(np.abs(G - R.T @ R)))

0.701847716177
